# Нейро-контентмейкер, помощник блогера | эксперта

## **Задача:**
Создать нейро-контентмейкера для генерации новых тем, контент-плана, постов, сценариев и других текстовых материалов в заданном стиле. Стиль переносится из постов блогера.

**Источник**:
Посты блогера, заранее подготовленные инструкции, собственная база знаний gpt моделей.

**Цель**:
*   Генерация контент-плана на 7 дней
*   Посты и сценарии для видео по заданной теме в стиле блогера

**Содержание работ**:

*   *Разработка алгоритма*:
 - выявление характеристик и получения стиля блогера.
 - создание контент-плана
 - создание постов и сценариев на основании подготовленного контент-плана
*   *Тестирование в процессе разработки*:
 - оценка характеристик стиля
 - оценка контент-плана
 - оценка постов и сценариев

**Ожидаемые результаты**:
Нейро-контентмейкер, генерирующий контент-план, посты и сценарии видео по заданной теме в стиле автора.


## Готовим окружение
Устанавливаем пакеты.
Подключаем библиотеки.
Задаем ключ API (openAI).
Задаем константы.

In [ ]:
#@title Устанавливаем пакеты
!pip install langchain langchain_community langchain-openai openai tiktoken faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.50
    Uninstalling langchain-core-0.3.50:
      Successfully uninstalled langchain-core-0.3.50
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchain
    Found

In [ ]:
#@title Импортируем библиотеки
import os
import re
import time
import json
import gdown
import getpass
import requests
import pathlib
import tempfile
import subprocess

import openai
from openai import OpenAI
import tiktoken

from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from tqdm.auto import tqdm
import textwrap as tw
from IPython.display import clear_output
from langchain.text_splitter import RecursiveCharacterTextSplitter
from google.colab import userdata

In [ ]:
# инициируем ключи OpenAI через секрет
key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = key

openai.api_key = key

In [ ]:
#@title Задаем константы
MODEL = "gpt-4o-mini"
temp = 0.3

post = 'https://docs.google.com/document/d/1HziKpcWIwmh60vKMu2PRfes9mjMxBqNp/edit?usp=sharing&ouid=114366634982817564013&rtpof=true&sd=true' # здесь посты блогера размеченные

max_tok = 2000 # Ограничение длины ответа

## Определяем пользовательские Функции

In [ ]:
# Функция форматирует строку по длинне для корректного вывода длинных строк
def wrap(s, max_width=150):
    wrapper = tw.TextWrapper(width=max_width, tabsize=4, replace_whitespace=False, drop_whitespace=True)
    return '\n'.join([wrapper.fill(l) for l in s.splitlines()])

# Функция форматирует строку удаляя ненужные знаки и символы для включения в результат
def format_result(text):
    text_mod = text.replace("- ", "").replace("*", "")
    return tw.fill(text_mod, 150)

# Функция загружает plane text из ГуглДока по URL-ссылке (url)
def load_googledoc_by_url(url: str) -> str:
        match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
        if match_ is None:
            raise ValueError('Invalid Google Docs URL')
        doc_id = match_.group(1)
        response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
        response.raise_for_status()
        return response.text

# Функция объединяет num_chunks элементов списка lst и возвращает новый список из объединенных элементов
def create_block_post(lst, num_chunks):
    block_list = []
    for i in range(0, len(lst), num_chunks):
        block_list.append(' '.join(lst[i:i+num_chunks]))
    return block_list

#функция добавления переходов на новую строку для удобства чтения
# здесь в параметр text подается текст без переносов строки
# функция добавит переходы на новую строку при достижении max_len символов
def insert_newlines(text: str, max_len: int = 160) -> str:
      words = text.split()
      lines = []
      current_line = ""
      for word in words:
          if len(current_line + " " + word) > max_len:
              lines.append(current_line)
              current_line = ""
          current_line += f' {word}'
      lines.append(current_line)
      return "\n".join(lines)

#функция добавления переходов на новую строку для удобства чтения
# здесь в параметр text подается текст, содержащий переносы строки
# функция добавит переходы на новую строку при достижении max_len символов
def insert_newlines_n(text: str, max_len: int = 150) -> str:
    lines = text.splitlines()
    new_lines = []
    for line in lines:
        words = line.split()
        current_line = ""
        for word in words:
            if len(current_line + " " + word) > max_len:
                new_lines.append(current_line)
                current_line = ""
            current_line += f' {word}'
        new_lines.append(current_line)
    return "\n".join(new_lines)

# функция подсчета токенов
def num_tokens_from_messages(messages, model):
      """Returns the number of tokens used by a list of messages."""
      try:
          encoding = tiktoken.encoding_for_model(model)
      except KeyError:
          encoding = tiktoken.get_encoding("cl100k_base")
      if model in ["gpt-4o", "gpt-o3-mini"]:  # note: future models may deviate from this
          num_tokens = 0
          for message in messages:
              num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
              for key, value in message.items():
                  num_tokens += len(encoding.encode(value))
                  if key == "name":  # if there's a name, the role is omitted
                      num_tokens += -1  # role is always required and always 1 token
          num_tokens += 2  # every reply is primed with <im_start> assistant
          return num_tokens
      else:
          raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.
          See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")

# Подсчет токенов в строке
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

# функция создания индексов БЗ
def create_search_index(file_text,
                        knowledge_base_link,
                        chunk_size=1024,
                        chunk_overlap=200):

    splitter = RecursiveCharacterTextSplitter(['\n\n', '\n', ' '], chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    count_tokens = 0
    source_chunks = []

    # разбиваем на несколько частей с помощью метода split_text
    for chunkID,chunk in enumerate(splitter.split_text(file_text)):
        source_chunks.append(Document(page_content=chunk, \
                            metadata={'source': knowledge_base_link,
                                      'chunkID': chunkID}))

    if len(source_chunks) > 0:
        db = FAISS.from_documents(source_chunks, OpenAIEmbeddings())
        count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")
        count_tokens += count_token
        print('Количество токенов в документе :', count_token)
        # print('ЦЕНА запроса:', 0.0004 * (count_token / 1000), ' $')

    print('\nЦЕНА запроса создания базы индексов:', 0.0004 * (count_tokens / 1000), ' $')
    return db


def answer_index(system, instructions, tema_posta, search_index, k):
    docs = search_index.similarity_search_with_score(tema_posta, k=k)
    message_content = '\n '.join([f'Отрывок текста №{i+1}\n{doc[0].page_content}' for i, doc in enumerate(docs)])
    message_content_bez_enter = message_content.replace('\n', ' ')
     # Удаляем от двухкратных до 20-кратных пробелов
    message_content_bez_enter2 = re.sub(r'\s{2,20}', ' ', message_content_bez_enter)
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": instructions + "Для написания поста тебе может понадобиться информация из этого документа: " + message_content_bez_enter2},
        ]

    completion = openai.chat.completions.create(model=MODEL, messages=messages, temperature=temp)
    return completion.choices[0].message.content

def gpt_traffic(system, topic, user, search_index, temp=0.5, k=7):
    docs = search_index.similarity_search(topic, k)
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'№{i+1}.' + doc.page_content for i, doc in enumerate(docs)]))
    messages = [
      {"role": "system", "content": system},
      {"role": "user", "content": f'{user}. Информация, которая сегодня может понадобиться: {message_content}. Не упоминай документ с информацией, указанный выше. Запрос пользователя: {topic}'}]
    completion = openai.chat.completions.create(model=MODEL, messages=messages, temperature=temp)
    answer = completion.choices[0].message.content
    return answer, message_content


## Загружаем посты
Забираем из гугл-документа посты блогера в переменную ( документ размечен: посты отделены друг от друга символами ##_)

In [ ]:
post_1 = load_googledoc_by_url(post)
post_1

'\ufeff##_\r\nПисатель, не желающий профессионально развиваться, к сожалению, превращается в графомана. Не желаю этого ни себе, ни друзьям по ремеслу. Поэтому ловите подборку из тридцати книг по писательскому мастерству:\r\n1. Роберт Макки «История на миллион». Хотя книга написана для сценаристов, её по праву можно назвать главным учебником по современной драматургии. Читать сложно, но обязательно, если хотите понять, как написать историю на миллион.\r\n2. Блейк Снайдер «Спасите котика». Книга обязательна для прочтения, если хотите, чтобы ваше произведение «держало читателя за горло».\r\n3. Милан Кундера «Искусство романа». Размышления о литературе одного из величайших романистов XX века.\r\n4. Айн Рэнд «Искусство беллетристики». Курс об искусстве беллетристики, прочитанный знаменитой писательницей в собственной гостиной в 1958 году, когда она находилась на пике творческой активности и была уже широко известна. Слушателями Айн Рэнд были два типа «студентов» — честолюбивые молодые писат

Формируем список из постов

In [ ]:
all_posts = post_1.split("##_")
print(len(all_posts))

31


Выведем содержание шестого поста

In [ ]:
all_posts[5]

'\r\n«Вдохновение – это такая гостья, которая не любит посещать ленивых»\r\nВысказывание Петра Ильича Чайковского.\r\nНадеюсь, представлять этого великого человека Творчества не нужно...😉\r\nВдохновлённых вам выходных🌺\r\n\r\n\r\n'

Очистим тексты от переходов на новую строку, чтобы вытягивал больше деталей + экономия токенов на незначащих символах

In [ ]:
all_posts_cleaned = [s.replace("\n", "") for s in all_posts]
print(len(all_posts_cleaned))

31


## Функции для выделения характеристик постов:

In [ ]:
#@title Выделение характеристик постов (модель GPT): анализирует в цикле посты и возвращает список, состоящий из характеристик, выявленных моделью
# в переменной promt - инструкция для system
# в переменной zapros_i - инструкция для user
# в переменной posts_list - список постов для анализа
def nomer_zaprosa(promt, zapros_i, posts_list):
    # формируем запрос к GPT
    zapros = zapros_i + '\n'.join(posts_list)
    messages = [
                {"role": "system", "content": promt},
                {"role": "user", "content": zapros}
              ]

    # получаем ответ GPT
    response = openai.chat.completions.create(model=MODEL,
                                              messages=messages,
                                              temperature=temp,
                                              max_tokens=max_tok)

    # возвращаем ответ GPT
    return response.choices[0].message.content

In [ ]:
#@title Обработка эмоциональных выражений, приветствий, вводных слов и обработка тем (обрабатываем не одиночные слова, а выражения. Для приветствий, вводных слов и эмоциональных выражений устанавливаем предельный размер 8 слов в выражении, для обработки тем постов берем выражения по 100 слов)

# функция форматирования списка фраз в очищенный от дубликатов список выражений
def clean_list(slova, target_len):   # target_len = 8 для всех выражений, кроме списка тем (= 100)
    list_r = slova.split('\n')

    cleaned_list = []       # в этот список сохраним выражения, очищенные от кавычек, точек, запятых и цифр
    for element in list_r:
        # Удаляем кавычки, запятые, точки и цифры
        cleaned_element = ''.join(filter(lambda x: x not in ['"', ',', '.', ')', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '-'], element))
        cleaned_list.append(cleaned_element)

    list_p = [x.strip() for x in cleaned_list] # Удаляем пробелы в начале и конце каждого значения (выражения)
    list_d = list(set(list_p))   # Удаляем дубликаты
    # sorted_list = sorted(list_d, key=lambda x: len(x)) # Сортируем по длине элементов (выражений)

    keywords = ["ранжированный", "список", "выражений", "выражениями"] #удалаяем те выражения, где есть эти слова
    list_с = [item for item in list_d if not any(keyword in item for keyword in keywords)]

    list_сс = list(filter(None, list_с)) # убираем пустые значения
    final_list = [item1 for item1 in list_сс if len(item1.split()) <= target_len]   #берем только выражения, где не больше target_len слов
    return final_list



In [ ]:
#@title Обработка смайликов с ранжированием по частотности. Параметр - список всех смайликов из постов. Очищаем этот список от пробелов и символов "‍" и "♂️". При помощи Counter из библиотеки collections определяем частоту вхождения каждого смайлика в список смайликов и ранжируем итоговый список по числу вхождений (по убыванию)
from collections import Counter

def process_smiley(smiley, all_posts):
    # получаем весь текст
    all_posts_text = '\n'.join(all_posts)

    #делим значения переменной через запятую и получаем список
    split_list = smiley.replace('\n', '').replace(' ', '').replace('\u200d', ',').split(',')

    # Очищаем список от пробелов и символов "‍" и "♂️"
    cleaned_list = [item.replace(" ", "").replace("‍", "").replace("♂️", "") for item in split_list]

    # Создаем объект Counter, который подсчитывает количество каждого элемента
    counter = [(smile, all_posts_text.count(smile)) for smile in cleaned_list]

    # Создаем ранжированный список по убыванию количества элементов
    ranked_list = sorted(counter, key=lambda x: x[1], reverse=True)
    print(ranked_list)
    new_list = [item[0] for item in ranked_list]

    return new_list

## Сущности (характеристики), которые нужно выделить из постов блогера
- вводные слова
- манера письма
- стиль
- лексика
- синтаксис
- эмоции и тональность
- синтаксические конструкции
- литературные приемы
- индивидуальные особенности
- психотип
- эмоциональные выражения
- смайлы и эмоджи

1. Вытягиваем характеристики из постов.
2. Обобщаем в одно общее описание для всех постов блогера.
3. Для каждой сущности сохраняем запрос и результат в отдельные переменные, по которым в дальнейшем программно сможем построить промпт для генерации постов.

Эту часть запроса будем включать в инструкцию для выделения характристик по каждой сущности:

promt_ = '''***Ты профессиональный главный редактор журнала.
Ты разговариваешь с блогером лицом к лицу, поэтому отвечаешь от второго лица.***'''.

Кроме этого для каждой сущности будем добавлять в инструкцию специфические для сущности промпты.

#### Вступительные (вводные) слова - выделяем с использованием модели GPT

In [ ]:
#00 запрос вводные слова
promt_ = '''Ты профессиональный главный редактор журнала.
Ты разговариваешь с блогером лицом к лицу, поэтому отвечаешь от второго лица.'''
promt_00 = promt_ + '''Ты очень точно умеешь находить в постах вводные слова, которые блогер использует в своих обращениях к читателям.'''
zapros_00 = '''
#1 Проанализируй мои посты и найди в них вводные слова (то есть слова, которые я использую в своих обращениях к читателям);
#2 Составь список этих вводных слов. ;
#3 Из составленного списка удали все приветствия, смайлики и эмодзи.
В качестве итогового результата выведи вводные слова через запятую.
Вот мои посты: '''

result_list_00 = nomer_zaprosa(promt_00, zapros_00, all_posts_cleaned) # вытягиваем
print (wrap(result_list_00))

Вот список вводных слов, которые вы используете в своих обращениях к читателям:

1. Поэтому
2. Например
3. Надеюсь
4. Дорогие
5. Желаю
6. Пусть
7. Вы
8. Как
9. Если
10. Извините
11. Вдохновляюсь
12. Напоминаю
13. Знаете
14. Помните
15. Хочу
16. Давайте
17. Это
18. Как-то
19. В общем
20. Да

Если вам нужно что-то еще, дайте знать!


Очищаем слова

In [ ]:
list_vvslov = clean_list(result_list_00, 8)
print("Слова:", tw.fill(', '.join(list_vvslov), 150))

Слова: Как, Надеюсь, Знаете, Пусть, Вы, Например, В общем, Вдохновляюсь, Дорогие, Както, Хочу, Помните, Напоминаю, Давайте, Желаю, Если, Извините, Это, Да,
Если вам нужно чтото еще дайте знать!, Поэтому


### Манера письма

In [ ]:
# @title Текст заголовка по умолчанию
#1 запрос манера
promt_1 = promt_ + '''Ты очень точно умеешь определять на основании постов манеру письма и речи блогера.'''
zapros_1 = '''Проанализируй мои посты и напиши вывод о свойственной для меня манере письма и речи.
В свой ответ включи текстовое описание длинной до 5 предложений.
Вот мои посты: '''

result_list_1 = nomer_zaprosa(promt_1, zapros_1, all_posts_cleaned) # вытягиваем
print (wrap(result_list_1))

Ваша манера письма и речи отличается яркой эмоциональностью и искренностью. Вы умело сочетаете личные размышления с практическими советами, что делает
ваши посты не только интересными, но и полезными для читателей. Ваш стиль легко воспринимается благодаря живым образам и метафорам, а также умению
делиться личным опытом, что создает атмосферу доверия и близости. Вы также активно используете вопросы и призывы к действию, что вовлекает аудиторию и
побуждает к взаимодействию. В целом, ваш стиль можно охарактеризовать как вдохновляющий и поддерживающий, что особенно важно в контексте писательского
сообщества.


### Стиль письма

In [ ]:
#2 запрос стиль
promt_2 = promt_ + '''Ты очень точно умеешь определять на основании постов стиль письма и речи блогера.'''
zapros_2 = '''Проанализируй мои посты и напиши вывод о свойственном мне стиле письма и речи.
В свой ответ включи текстовое описание длинной до 5 предложений.
Вот посты: '''

result_list_2 = nomer_zaprosa(promt_2, zapros_2, all_posts_cleaned) # вытягиваем
print (wrap(result_list_2))

Ваш стиль письма и речи можно охарактеризовать как искренний, вдохновляющий и насыщенный личными размышлениями. Вы умело сочетаете элементы личного
опыта с практическими советами, что делает ваши посты не только интересными, но и полезными для читателей. Ваша речь наполнена позитивом и оптимизмом,
вы часто используете метафоры и образы, что придаёт текстам живость и эмоциональную окраску. Вы также активно делитесь своими увлечениями и
литературными находками, что демонстрирует вашу страсть к писательству и желанию помочь другим в их творческом пути. В целом, ваш стиль можно описать
как дружелюбный, поддерживающий и вдохновляющий, что делает вас близким и понятным для вашей аудитории.


### Лексика

In [ ]:
#3 запрос свойственная блоггеру лексика
promt_3 = promt_ + '''Ты очень точно умеешь определять на основании постов лексику, характерную блогеру.'''
zapros_3 = '''Проанализируй мои посты и напиши вывод о свойственной мне лексике.
В свой ответ включи текстовое описание длинной до 5 предложений.
Вот мои посты: '''

result_list_3 = nomer_zaprosa(promt_3, zapros_3, all_posts_cleaned) # вытягиваем
print (wrap(result_list_3))

На основании ваших постов можно выделить несколько характерных черт вашей лексики и стиля. Вы используете разнообразные литературные приемы, что
делает ваши тексты живыми и эмоциональными. Ваша лексика насыщена образами, метафорами и личными размышлениями, что создает ощущение близости и
искренности. Вы часто обращаетесь к читателю, используя дружеский и теплый тон, что делает ваши сообщения доступными и понятными. Кроме того, вы
активно делитесь своими мыслями о писательском мастерстве и личном опыте, что подчеркивает вашу страсть к литературе и стремление к саморазвитию.


### Синтаксис
Здесь на анализ будем передавать список постов, которые не очищали от переносов строк (потому что перенос строки - это тоже элемент синтаксиса, который нужно проанализировать)

In [ ]:
#4 запрос особенности синтаксиса
promt_4 = promt_ + '''Ты очень точно умеешь определять на основании постов блогера синтаксис постов.'''
zapros_4 = '''Проанализируй мои посты и напиши вывод о свойственном мне синтаксисе.
В свой ответ включи текстовое описание длинной до 5 предложений.
Вот посты: '''

result_list_4 = nomer_zaprosa(promt_4, zapros_4, all_posts) # вытягиваем Здесь в третьем параметре список постов без удаления переходов на новую строку
print (wrap(result_list_4))

Ваш синтаксис постов характеризуется яркой эмоциональностью и живостью. Вы используете множество метафор и образных выражений, что делает текст
насыщенным и увлекательным для чтения. Часто обращаетесь к читателю, создавая ощущение личного общения и вовлеченности. Ваша структура постов
разнообразна: от кратких заметок о повседневной жизни до глубоких размышлений о литературе и саморазвитии, что придает вашему стилю многогранность. В
целом, ваш стиль можно охарактеризовать как вдохновляющий и позитивный, с акцентом на личные переживания и наблюдения.


### Эмоции и тональность
Здесь анализируем эмоциональную окраску и тональность постов

In [ ]:
# 5 запрос эмоции
promt_5 = promt_ + '''Ты очень точно умеешь определять на основании постов блогера эмоциональную окраску постов.'''
zapros_5 = '''Проанализируй мои посты и напиши вывод о свойственных мне эмоциональной окраске и тональности.
В свой ответ включи текстовое описание длинной до 5 предложений.
Вот мои посты: '''

result_list_5 = nomer_zaprosa(promt_5, zapros_5, all_posts_cleaned) # вытягиваем
print (wrap(result_list_5))

В твоих постах прослеживается яркая эмоциональная окраска, наполненная оптимизмом, вдохновением и стремлением к самосовершенствованию. Ты активно
делишься своими мыслями о писательском мастерстве, личных переживаниях и радостях повседневной жизни, что создает ощущение близости и искренности.
Тональность твоих текстов варьируется от серьезного анализа до легкого и игривого стиля, что делает их разнообразными и интересными для читателя. Ты
часто подчеркиваешь важность маленьких радостей и чудес, что создает атмосферу позитива и надежды. В целом, твои посты вдохновляют на размышления и
побуждают к действию, что делает их ценными для аудитории.


### Синтаксическая конструкция
Здесь для анализа опять подаем в gpt посты, которые не очищали от переносов строки

In [ ]:
#6 запрос  синтаксическая структура
promt_6 = promt_ + '''Ты очень точно умеешь определять на основании постов блогера
синтаксическую длину предложений поста, особенности: структуры текста,
структуры текста с использованием списков, синтаксической структуры предложений, структуры текста по абзацам.'''
zapros_6 = '''Проанализируй мои посты и напиши вывод о синтаксической длине предложений, об особенностях:
структуры текста, структуры текста с использованием списков, синтаксической структуры предложений,
структуры текста по абзацам.
В свой ответ включи текстовое описание длинной до 10 предложений.
Вот мои посты: '''

result_list_6 = nomer_zaprosa(promt_6, zapros_6, all_posts) # вытягиваем
print (wrap(result_list_6))

Ваши посты демонстрируют разнообразие стилей и подходов к написанию, что делает их интересными и многогранными.

1. **Синтаксическая длина предложений**: В большинстве постов вы используете как короткие, так и длинные предложения, что создает динамику текста.
Короткие предложения часто используются для акцентирования внимания на важных моментах, в то время как длинные позволяют более подробно раскрыть
мысль.

2. **Структура текста**: Ваши посты имеют четкую структуру, которая включает вводные части, основное содержание и заключение. Это помогает читателям
легко следовать за вашими мыслями. В некоторых постах вы используете списки, что делает информацию более доступной и организованной.

3. **Структура текста с использованием списков**: Например, в посте о 10-дневной программе «промывки мозга» вы применяете списки для представления
шагов, что позволяет читателям быстро усвоить информацию и сделать заметки.

4. **Синтаксическая структура предложений**: Вы часто используете разнообразные 

### Литературные приемы

In [ ]:
#7 запрос литературные приемы
promt_7 = promt_ + '''Ты очень точно умеешь определять на основании постов блогера использование литературных приемов. '''
zapros_7 = '''Проанализируй мои посты и напиши вывод о свойственных мне литературных приемах.
В свой ответ включи текстовое описание длинной до 5 предложений.
Вот мои посты: '''

result_list_7 = nomer_zaprosa(promt_7, zapros_7, all_posts_cleaned) # вытягиваем
print (wrap(result_list_7))

В ваших постах заметны несколько характерных литературных приемов, которые делают ваш стиль уникальным и привлекательным. Во-первых, вы активно
используете метафоры и образы, что придаёт вашим текстам поэтичность и глубину. Например, сравнения природы с внутренними состояниями создают
атмосферу уюта и гармонии. Во-вторых, вы применяете элементы личного повествования, что позволяет читателю лучше понять ваши эмоции и переживания,
создавая ощущение близости и доверия. Также в ваших текстах присутствует диалог с читателем, что делает их интерактивными и вовлекает аудиторию в
обсуждение. Наконец, вы часто используете юмор и иронию, что добавляет лёгкости и делает чтение приятным.


### Индивидуальные особенности блогера

In [ ]:
#8 запрос индивидуальные особенности блогера
promt_8 = promt_ + '''Ты очень точно умеешь определять на основании постов индивидуальные особенности блогера. '''
zapros_8 = '''Проанализируй мои посты и напиши вывод о свойственных мне индивидуальных особенностях, конкретизируя их.
В свой ответ включи текстовое описание длинной до 5 предложений.
Вот мои посты: '''

result_list_8 = nomer_zaprosa(promt_8, zapros_8, all_posts) # вытягиваем без очистки от переводов строки
print (wrap(result_list_8))

На основе ваших постов можно выделить несколько ключевых индивидуальных особенностей. Во-первых, вы проявляете глубокую страсть к писательству и
литературе, что видно из ваших рекомендаций книг по писательскому мастерству и размышлений о процессе создания текстов. Во-вторых, вы цените мелочи и
находите радость в повседневной жизни, что отражается в ваших описаниях простых удовольствий, таких как прогулки на свежем воздухе или общение с
детьми. В-третьих, вы стремитесь к саморазвитию и осознанности, что проявляется в ваших размышлениях о здоровье, счастье и внутреннем состоянии.
Наконец, вы активно делитесь своими мыслями и опытом с другими, что говорит о вашем желании вдохновлять и поддерживать окружающих. Эти качества делают
вас не только интересным блогером, но и человеком, способным влиять на других своим позитивным подходом к жизни.


### Психотип (определяем психотип блогера)

In [ ]:
# 9 запрос психотип
promt_9 = promt_ + '''Ты очень точно умеешь определять на основании постов психотип блогера по соционике. '''
zapros_9 = '''Проанализируй мои посты и напиши вывод к какому психотипу по соционике можно меня отнести
на основании моих постов.
В свой ответ включи текстовое описание длинной до 5 предложений.
Вот мои посты: '''

result_list_9 = nomer_zaprosa(promt_9, zapros_9, all_posts_cleaned) # вытягиваем
print (wrap(result_list_9))

На основании ваших постов можно сделать вывод, что вы относитесь к психотипу "Интуитивный логик" (ILI) по соционике. Ваши тексты полны глубоких
размышлений о жизни, творчестве и личностном развитии, что указывает на вашу способность видеть более широкую картину и находить связи между
различными аспектами жизни. Вы активно делитесь знаниями и опытом, стремитесь к самосовершенствованию и вдохновляете других, что также свидетельствует
о вашей интуитивной природе. Ваши посты наполнены эмоциональной окраской и личными переживаниями, что говорит о развитом эмоциональном интеллекте и
способности к эмпатии. Вы ищете гармонию и красоту в повседневной жизни, что делает вас чутким и отзывчивым человеком.


### Эмоциональные выражения
выделяем список эмоциональных выражений, которые использует блоггер

In [ ]:
#10 запрос слова
promt_10 = promt_ + '''Ты очень точно умеешь определять на основании постов свойственные блогеру эмоциональные выражения.'''
zapros_10 = '''
Давай действовать по шагам.
1. Проанализируй мои посты и составь ранжированный список из 15 наиболее характерных для моих постов эмоциональных выражений (например, "ну разве не круто?", "будет огонь", "такой кайф!").
2. Исключи из ранжированного списка все смайлики и эмоджи.
Выведи очищенный ранжированный список свойственных мне эмоциональных выражений.
Не добавляй комментарии.
Вот мои посты: '''

result_list_10 = nomer_zaprosa(promt_10, zapros_10, all_posts_cleaned) # вытягиваем
print (wrap(result_list_10))

1. кайф
2. чудо
3. радость
4. вдохновение
5. счастье
6. удовольствие
7. легкость
8. волшебство
9. замечательно
10. прекрасно
11. удивительно
12. здорово
13. приятно
14. замечательно
15. обожаю


### Cмайлики
Выделяем из постов блоггера список наиболее употребляемых им смайликов и эмоджи

In [ ]:
#11 запрос эмодзи
promt_11 = promt_ + '''Ты очень точно умеешь определять на основании постов свойственные блогеру смайлики и эмодзи.'''
zapros_11 = '''
Проанализируй мои посты и напиши  ранжированный список через запятую из 10 наиболее характерных для моих постов смайликов и эмодзи.
(Пример ответа: 🔥, 🚀, 👉, ❗️, 🎁 Конец примера). Запрещено в ответ включать описание смайликов и эмодзи.
Вот мои посты: '''

result_list_11 = nomer_zaprosa(promt_11, zapros_11, all_posts_cleaned) # вытягиваем

# сортируем по частоте встречаемости в постах
result_list_11_sorted = process_smiley(result_list_11, all_posts)

print("Смайлы", result_list_11_sorted)

[('📌', 11), ('✨', 8), ('🤗', 5), ('💖', 3), ('😊', 3), ('♀️', 3), ('🌺', 2), ('🤷', 2), ('🔥', 1), ('📖', 1), ('🎉', 0)]
Смайлы ['📌', '✨', '🤗', '💖', '😊', '♀️', '🌺', '🤷', '🔥', '📖', '🎉']


### ТОП 20 словосочетаний
выделем наиболее часто используемые блоггером словосочетания

In [ ]:
#10 запрос слова
promt_12 = promt_ + '''Ты очень точно умеешь определять на основании постов наиболее часто используемые блогером словосочентания и выражения.'''
zapros_12 = '''
1. Проанализируй мои посты и составь ранжированный список из 20 наиболее часто используемых мной словосочетаний и выражений,
2. Исключи из ранжированного списка все смайлики и эмодзи.
В свой ответ включи только очищенный ранжированный список.
Вот мои посты: '''

result_list_12 = nomer_zaprosa(promt_12, zapros_12, all_posts_cleaned) # вытягиваем
print (wrap(result_list_12))

1. Писательское мастерство
2. Чудеса и чудесатости
3. Маленькие радости
4. Вдохновение и творчество
5. Путешествия по России
6. Книги по писательскому мастерству
7. Здоровье и счастье
8. Программа промывки мозга
9. Осознанное ничегонеделание
10. Взаимоотношения и общение
11. Внутренний мир
12. Закон дарения
13. Эмоции и чувства
14. Успехи детей
15. Обновление и легкость
16. Позитивное мышление
17. Творческий процесс
18. Секреты счастья
19. Психология и саморазвитие
20. Визуализация и намерения


###Выведем обобщения (`promt_itog`) для всех характеристик.




In [ ]:
promt_n = 'Ты блогер. Твоя задача писать свои посты.......\n\n\n '
promt_1 = '\n\nМанера письма\n ' + result_list_1.replace('\n', '')
promt_2 = '\n\nСтиль письма\n' + result_list_2.replace('\n', '')
promt_3 = '\n\nЛексика\n' + result_list_3.replace('\n', '')
promt_4 = '\n\nСинтаксис\n' + result_list_4.replace('\n', '')
promt_5 = '\n\nЭмоциональная окраска и тональность\n' + result_list_5.replace('\n', '')
promt_6 = '\n\nСтруктура постов\n' + result_list_6.replace('\n', '')
promt_7 = '\n\nЛитературные приемы\n' + result_list_7.replace('\n', '')
promt_8 = '\n\nИндивидуальные особенности\n' + result_list_8.replace('\n', '')
promt_9 = '\n\nПсихотип\n' + result_list_9.replace('\n', '')
promt_00 = '\n\nВводные слова\n' + result_list_00
promt_10 = '\n\nЭмоциональные выражения\n' + result_list_10
promt_11 = '\n\nСмайлики и эмодзи\n' + result_list_11.replace('\n', '')
promt_itog = promt_n + promt_1 + promt_2 + promt_3 + promt_4 + promt_5 + promt_6 + promt_7 + promt_8 + promt_9 + promt_00 + promt_10 + promt_11

print(wrap(promt_itog))

Ты блогер. Твоя задача писать свои посты.......




Манера письма
 Ваша манера письма и речи отличается яркой эмоциональностью и искренностью. Вы умело сочетаете личные размышления с практическими советами, что
делает ваши посты не только интересными, но и полезными для читателей. Ваш стиль легко воспринимается благодаря живым образам и метафорам, а также
умению делиться личным опытом, что создает атмосферу доверия и близости. Вы также активно используете вопросы и призывы к действию, что вовлекает
аудиторию и побуждает к взаимодействию. В целом, ваш стиль можно охарактеризовать как вдохновляющий и поддерживающий, что особенно важно в контексте
писательского сообщества.

Стиль письма
Ваш стиль письма и речи можно охарактеризовать как искренний, вдохновляющий и насыщенный личными размышлениями. Вы умело сочетаете элементы личного
опыта с практическими советами, что делает ваши посты не только интересными, но и полезными для читателей. Ваша речь наполнена позитивом и оптимизмом,
вы часто

## Эксперименты

###Эксперимент 1. Контент-план

In [ ]:
promt_itog_1 = (
    promt_itog +
    '''

\n\nДополнительные инструкции:
Ты – гениальный контент-менеджер, специализирующийся на экспертных и личных блогах:
- Создай подробный контент-план для соцсетей на 7 дней, где каждый день отмечен как рабочий или выходной, используй схему: 5 будних дней и 2 выходных.
- Сгенерируй 3 релевантные рубрики и чередуй их между собой.
- Предложи 7 уникальных идей для постов, где каждая идея соответствует одной из рубрик.
- Используй разные категории контента: экспертный (от 30%), обзор трендов, юмористический, продающий (от 20%), неожиданный ракурс темы.
- Укажи точное оптимальное время публикации по Москве для каждого дня.
- Опиши, на кого направлен пост (целевая аудитория).
- Укажи, какой формат постов должен использоваться (текст, изображение, видео, инфографика и т.п.) и, если необходимо, для каких соцсетей.
- Определи СТА – какое действие должен совершить пользователь после прочтения поста.
Выходной результат должен включать день недели (будний/выходной), рубрику, идею поста, оптимальное время публикации по Москве,
целевую аудиторию, формат контента, призыв к действию

'''
)

zapros = '''
Напиши контент-план для моего телеграм канала на 7 дней'''
messages = [
      {"role": "system", "content": promt_itog},
      {"role": "user", "content": zapros}
      ]

response_1 = openai.chat.completions.create(model=MODEL,
                                                messages=messages,
                                                temperature=0.4,
                                                max_tokens=2000)

print(wrap(response_1.choices[0].message.content))


Конечно! Вот пример контент-плана для вашего телеграм-канала на 7 дней. Каждый день включает в себя разные форматы контента, чтобы поддерживать
интерес вашей аудитории и вовлекать их в обсуждения.

### Контент-план на 7 дней

**День 1: Вдохновение понедельника**
- **Пост:** Поделитесь цитатой о писательстве или саморазвитии, которая вас вдохновляет. Напишите, почему она важна для вас.
- **Вопрос к читателям:** Какую цитату вы бы добавили в свою коллекцию вдохновения?

**День 2: Литературные находки**
- **Пост:** Рекомендация книги по писательскому мастерству. Опишите, что именно вам понравилось в этой книге и как она повлияла на ваш стиль.
- **Призыв к действию:** Попросите читателей поделиться своими любимыми книгами о писательстве.

**День 3: Творческий процесс**
- **Пост:** Расскажите о своем текущем проекте или процессе написания. Поделитесь своими мыслями и переживаниями.
- **Вопрос к читателям:** Какие трудности вы сталкиваетесь в своем творчестве?

**День 4: Полезные советы**
- 

###Эксперимент 2. Пост

In [ ]:
promt_itog_2 = (
    promt_itog +
    '''

\n\nДополнительные инструкции:
- Пиши короткими абзацами.
- Используй риторические вопросы.
- Начинай пост с цепкого, эмоционального хука.
- Активно применяй психологические триггеры (нейрокопирайтинг).
- Используй метод скользкой горки при построении текста: сильное начало: зацепи внимание (необычный факт, история, провокация, вопрос),
кратко и динамично подай основную мысль, предупредите сомнения, сразу отвечай на возможные вопросы читателей.
- Используй эффективный сторителлинг с героем, конфликтом и клиффхэнгерами.
- Заверши пост на яркой или провокационной ноте, без стандартного вывода. Или дай призыв к действию: чётко сформулируй, что нужно сделать дальше.
- Активно вовлекай читателя эмоционально и визуально (смайлики, эмоции).
- В конце поста добавь по 5 самых релевантных хештегов для увеличения охватов на русском и английском языках
- Отредактируй текст и исправь лексические ошибки (такие как однокоренные или одинаковые слова в соседних предложениях,
убери местоимения принадлежности: свой, своя, ваш, твой и т.п. и устрани другие лексические ошибки).
- Очень важно сохранить стиль и синтаксис блогера, включая типичные приветствия, смайлики и эмодзи.

'''
)


zapros = '''
Напиши пост на тему: "5 лучших техник для развития воображения".
'''
messages = [
      {"role": "system", "content": promt_itog},
      {"role": "user", "content": zapros}
      ]

response_2 = openai.chat.completions.create(model=MODEL,
                                                messages=messages,
                                                temperature=0.3,
                                                max_tokens=1000)

print(wrap(response_2.choices[0].message.content))


Дорогие друзья! 🌺

Сегодня я хочу поделиться с вами чем-то поистине волшебным — техниками, которые помогут развить ваше воображение! ✨ Воображение — это ключ к
творчеству, новым идеям и вдохновению. Оно позволяет нам видеть мир не только таким, какой он есть, но и таким, каким он может быть. Итак, давайте
погрузимся в этот удивительный процесс!

### 1. Визуализация
Представьте, что вы находитесь в своем идеальном месте. Это может быть пляж, лес или уютное кафе. Закройте глаза и постарайтесь максимально детально
представить, как выглядит это место, какие звуки его окружают, какие запахи вы чувствуете. Эта техника не только развивает воображение, но и помогает
расслабиться и зарядиться энергией. 🌊🌳

### 2. Чтение художественной литературы
Чтение — это не просто способ провести время, это настоящая тренировка для вашего воображения! 📖 Погружаясь в мир книг, вы создаете образы,
представляете персонажей и события. Попробуйте читать разные жанры, чтобы расширить свои горизонты и вдохновиться

###Эксперимент 3. Сценарий Reels

In [ ]:
promt_itog_3 = (
    promt_itog +
    '''

\n\nДополнительные инструкции для сценария Reels:
- Начни с цепляющего кадра или фразы (вопрос, интрига, провокация). Первые 2-3 секунды должны моментально привлечь внимание.
- Структурируй ролик: А. Зацепка (0-3 сек.) Эмоциональный или неожиданный вопрос, факт, ситуация. Б. Развитие темы (3-15 сек.)Раскрой кратко и ясно основной посыл.
Дай конкретику, цифры, факты. В. Работа с возражениями (15-20 сек.) Предупреди основные сомнения зрителя, убеди в реальности результата. Г. Итог и призыв к действию (20-30 сек.)
Чёткий, понятный и эмоциональный call-to-action.
- Используй сторителлинг: Короткая история с героем и конфликтом. Добавь эмоциональность, юмор или драму. Усиль вовлечённость через неожиданный поворот или инсайт.
Пример короткой истории: «Когда я впервые запустил свой онлайн-курс, никто его не купил. Я чуть не бросил всё, пока не узнал о простом лайфхаке…»
- Используй короткие, выразительные реплики — одна мысль = один кадр.
- Усиль сценарий психологическими триггерами (ощущение срочности, социальные доказательства, авторитет и т.п.)
- Сделай акценти на визуальном ряду: смена кадров каждые 2-3 секунды для удержания внимания, титры или текст для ключевых моментов.
- Добавляй смайлики и визуальные акценты, если текст будет озвучен или наложен субтитрами.
- Делай акцент на эмоции: удиви, вдохнови, взбей, заставь почувствовать!
- В конце обязательно должен быть призыв: лайк, подписка, сохранение, вопрос в комментарии.
- Предложи идеи для музыкального сопровождения видео.

'''
)

zapros = '''
Напиши сценарий Reels на тему: "Игра с ассоциациями".'''
messages = [
      {"role": "system", "content": promt_itog},
      {"role": "user", "content": zapros}
      ]

response_3 = openai.chat.completions.create(model=MODEL,
                                                messages=messages,
                                                temperature=0.5,
                                                max_tokens=1300)

print(wrap(response_3.choices[0].message.content))


🎥 **Сценарий Reels: "Игра с ассоциациями"** 🎥

**[Кадр 1: Вступление]**
*(Вы находитесь в уютном уголке, может быть, с книгами или чашкой чая в руках.)*

🗣️ **Вы:** Привет, дорогие друзья! Сегодня я предлагаю вам поиграть в увлекательную игру с ассоциациями! 🎉

---

**[Кадр 2: Объяснение правила]**
*(Крупный план на ваше лицо, с энергией и улыбкой.)*

🗣️ **Вы:** Правила просты: я называю слово, а вы в комментариях пишете, какие ассоциации оно у вас вызывает! 💭✨

---

**[Кадр 3: Первое слово]**
*(Вы берете листок бумаги или используете карточки.)*

🗣️ **Вы:** Итак, первое слово — "лето"! 🌞 Какое первое слово приходит вам на ум? Пишите в комментариях! 👇

*(На экране появляется текст: "Лето" с ярким фоном.)*

---

**[Кадр 4: Ваша ассоциация]**
*(Вы показываете свою ассоциацию — например, картинку с пляжем или мороженым.)*

🗣️ **Вы:** А у меня ассоциация с "летом" — "море"! 🌊 Обожаю ощущение песка под ногами и свежий ветерок! 💖

---

**[Кадр 5: Второе слово]**
*(Вы снова берете карточку.)*